<a href="https://colab.research.google.com/github/JSJeong-me/AI-Innovation-2024/blob/main/RL/6-6-StateFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install autogen

In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [3]:
import tempfile

from autogen.coding import LocalCommandLineCodeExecutor

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
temp_dir = tempfile.TemporaryDirectory()
executor = LocalCommandLineCodeExecutor(
    timeout=1200,  # Timeout for each code execution in seconds.
    work_dir=temp_dir.name,  # Use the temporary directory to store the code files.
)

gpt4_config = {
    "cache_seed": False,  # change the cache_seed for different trials
    "temperature": 0,
    "config_list": [{"model": "gpt-4", "api_key": os.environ.get("OPENAI_API_KEY")}],
    "timeout": 1200,
}

In [5]:
import autogen
import os
from dotenv import load_dotenv

#첫 대화를 실행하는 에이전트
initializer = autogen.UserProxyAgent(
    name="Init",
    code_execution_config=False,
    human_input_mode="NEVER"
)

# 첫 번째 코드 실행기 에이전트
executor1 = autogen.UserProxyAgent(
    name="Executor1",
    human_input_mode="NEVER",
    code_execution_config={"executor": executor},
)

# 두 번째 코드 실행기 에이전트
executor2 = autogen.UserProxyAgent(
    name="Executor2",
    human_input_mode="NEVER",
    code_execution_config={"executor": executor},
)

# GPT 모델을 사용하는 감시 에이전트
monitor = autogen.AssistantAgent(
    name="Monitor",
    llm_config=gpt4_config,
    system_message="당신은 코드 실행을 감시하고 결과를 보고하는 모니터입니다. 각 실행기의 결과를 분석하고 요약하여 보고해주세요."
)
def state_transition(last_speaker, groupchat):
    if last_speaker == initializer:
        return executor1
    elif last_speaker == executor1:
        return executor2
    elif last_speaker == executor2:
        return monitor
    elif last_speaker == monitor:
        return None  # 모니터 이후에 대화 종료
    else:
        return None

# 그룹 채팅 매니저 설정
groupchat = autogen.GroupChat(agents=[initializer, executor1, executor2, monitor], messages=[], max_round=10, speaker_selection_method=state_transition)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=gpt4_config)




In [6]:
# 코드 블록 정의
code_block = """
print(f"테스트")
"""
# initializer가 채팅 시작
initializer.initiate_chat(
    manager,
    message=f"코드 실행 테스트를 시작합니다. Executor1과 Executor2가 순차적으로 다음 코드를 실행하고, Monitor가 결과를 분석할 것입니다. \n코드실행기는 다음 코드를 실행하세요.:\n```python\n{code_block}\n```"
)

# Executor1에게 코드 실행 요청
executor1.send(
    message=f"다음 코드를 실행하고 결과를 보고해주세요:\n```python\n{code_block}\n```",
    recipient=manager
)

# Executor2에게 코드 실행 요청
executor2.send(
    message=f"다음 코드를 실행하고 결과를 보고해주세요:\n```python\n{code_block}\n```",
    recipient=manager
)

# Monitor에게 결과 분석 요청
monitor.send(
    message="Executor1과 Executor2의 코드 실행 결과를 분석하고 보고해주세요.",
    recipient=manager
)





Init (to chat_manager):

코드 실행 테스트를 시작합니다. Executor1과 Executor2가 순차적으로 다음 코드를 실행하고, Monitor가 결과를 분석할 것입니다. 
코드실행기는 다음 코드를 실행하세요.:
```python

print(f"테스트")

```

--------------------------------------------------------------------------------

Next speaker: Executor1


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
Executor1 (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 테스트


--------------------------------------------------------------------------------

Next speaker: Executor2


>>>>>>>> EXECUTING CODE BLOCK (inferred language is python)...
Executor2 (to chat_manager):

exitcode: 0 (execution succeeded)
Code output: 테스트


--------------------------------------------------------------------------------

Next speaker: Monitor

Monitor (to chat_manager):

모든 실행기가 코드를 성공적으로 실행했습니다. 코드의 출력은 "테스트"입니다. 에러나 예외는 발생하지 않았습니다.

--------------------------------------------------------------------------------
Executor1 (to chat_manager):

다음 코드를 실행하고 결과를 보고해주세요

In [7]:
# # Executor1에게 코드 실행 요청
# executor1.send(
#     f"다음 코드를 실행해주세요:\n```python\n{code_block}\n```",
#     manager
# )

# # Executor2에게 코드 실행 요청
# executor2.send(
#     f"Executor1의 실행 결과를 확인하고, 다음 코드를 실행해주세요:\n```python\n{code_block}\n```",
#     manager
# )

# # 모니터에게 결과 분석 요청
# monitor.send(
#     "두 실행기의 코드 실행 결과를 분석하고 보고해주세요.",
#     manager
# )